In [5]:
def remove_and_add_text_values(file_path, output_file_path):
    with open(file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        lines = infile.readlines()

        # Function 1: Remove lines starting with "indicator_Order_"
        lines = [line for line in lines if not line.startswith('indicator_Order_')]

        # Function 2: Add new indicator_Order attributes corresponding to the indicator_Text attribute 
        indicator_number = 1
        for line in lines:
            if line.startswith("indicator_Text_"):
                # Add a new line with the desired text
                outfile.write(f'indicator_Order_{indicator_number}:{indicator_number}\n{line}')
                indicator_number += 1
            elif line == "\n":  # Check for a new section (empty line)
                indicator_number = 1  # Reset the indicator number for a new section
                outfile.write(line)
            else:
                outfile.write(line)

    # Call the new function to add quotes to PreUnit and PostUnit values
    add_pre_and_postunit_quotes(output_file_path)

def add_pre_and_postunit_quotes(input_file):
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    with open(input_file, 'w') as outfile:
        for line in lines:
            if "PreUnit:" in line or "PostUnit:" in line:
                # Find the index of the colon
                colon_index = line.index(":")
                # Extract the value after the colon
                value = line[colon_index + 1:].strip()
                # Put the value into quotation marks
                line = line[:colon_index + 1] + f'"{value}"\n'
            outfile.write(line)

# Specify your file paths
file_path = "/Users/Documents/Main/Postprocessing/Merged_models.txt"
output_file_path = "/Users/Documents/Main/Postprocessing/Final_Merged_Model.txt"

# Example usage:
remove_and_add_text_values(file_path, output_file_path)

In [6]:
#rearrange attributes

class Section:
    def __init__(self, lines):
        self.lines = lines

    def __hash__(self):
        return hash(tuple(sorted(self.lines)))

    def __eq__(self, other):
        return sorted(self.lines) == sorted(other.lines)

    @staticmethod
    def remove_prefix(line, prefixes):
        for prefix in prefixes:
            line = line.replace(prefix, "")
        return line.strip()

    def process_lines(self, prefixes):
        processed_lines = [self.remove_prefix(line, prefixes) for line in self.lines]
        return processed_lines

# Define the file path
file_path = "/Users/noahritfeld/Documents/Main/Postprocessing/Final_Merged_Model.txt"

# Read the merged data from the specified file
with open(file_path, "r") as file:
    merged_data = file.read()

# Split merged data into seperated fragments again
separate_fragments = merged_data.strip().split("\n\n")

# Initialize sets to store unique sections for each category
unique_topics = set()
unique_indicators = set()
unique_questions = set()

# Define the prefixes to be removed
prefixes = ['topic_', 'indicator_', 'question_']

# Process each combined fragments
for separate_fragment in separate_fragments:
    # Split subsections of the combined fragments
    subsections = separate_fragment.strip().split("\n")
    # Process each separated fragment and create Section objects
    topic, indicator, question = [], [], []
    for subsection in subsections:
        prefix, data = subsection.split(":", 1)
        if prefix.startswith("topic_"):
            topic.append(f"{prefix}:{data.strip()}")
        elif prefix.startswith("indicator_"):
            indicator.append(f"{prefix}:{data.strip()}")
        elif prefix.startswith("question_"):
            question.append(f"{prefix}:{data.strip()}")
    # Create Section objects and add them to respective sets
    topic_section = Section(topic)
    indicator_section = Section(indicator)
    question_section = Section(question)

    # Process lines and add to respective sets
    unique_topics.add(Section(topic_section.process_lines(prefixes)))
    unique_indicators.add(Section(indicator_section.process_lines(prefixes)))
    unique_questions.add(Section(question_section.process_lines(prefixes)))

# Open the same file in write mode to overwrite its content
with open(file_path, "w") as file:
    file.write("Topics:\n")
    for topic in unique_topics:
        file.write("\n".join(topic.lines) + "\n\n")
    file.write("Indicators:\n")
    for indicator in unique_indicators:
        file.write("\n".join(indicator.lines) + "\n\n")
    file.write("Questions:\n")
    for question in unique_questions:
        file.write("\n".join(question.lines) + "\n\n")

In [7]:
import re

def remove_order_and_text_suffix(input_file_path):
    # Read data from the input file
    with open(input_file_path, 'r') as file:
        data = file.read()

    # Define a regular expression pattern to match and replace the Order keys and values
    order_pattern = r'Order_(\d+):'
    modified_data_order = re.sub(order_pattern, r'Order:', data)

    # Define a regular expression pattern to match and replace the Text keys and values
    text_pattern = r'Text_(\d+):'
    modified_data_text = re.sub(text_pattern, r'Text:', modified_data_order)

    # Open the same file in write mode to overwrite its content
    with open(input_file_path, "w") as output_file:
        output_file.write(modified_data_text)

    print(f"Modified data written to '{input_file_path}'")

# Example usage
input_file_path = "/Users/Documents/Main/Postprocessing/Final_Merged_Model.txt"
remove_order_and_text_suffix(input_file_path)


Modified data written to '/Users/noahritfeld/Documents/Main/Postprocessing/Final_Merged_Model.txt'


In [8]:
def add_merged_introduction(input1_path, input2_path, output3_path):
    # Read content from DSL_Model_A.txt
    with open(input1_path, 'r') as input1_file:
        input1_content = input1_file.read()

    # Extract Name and Description from DSL_Model_A.txt
    name_input1 = input1_content.split('\n')[0].strip().split(':')[-1].strip(' "')
    description_input1 = input1_content.split('\n')[3].strip().split(':')[-1].strip(' "')

    # Read content from DSL_Model_B.txt
    with open(input2_path, 'r') as input2_file:
        input2_content = input2_file.read()

    # Extract Name and Description from DSL_Model_B.txt
    name_input2 = input2_content.split('\n')[0].strip().split(':')[-1].strip(' "')
    description_input2 = input2_content.split('\n')[3].strip().split(':')[-1].strip(' "')

    # Combine Name and Description
    merged_name = f'Name: "Merged method of {name_input1} and {name_input2}"'
    merged_description = f'"This is a description about {description_input1} and {description_input2}"'

    # Create the merged introduction section
    merged_introduction = f"{merged_name}\nVersion:4.0\nisPublic:true\nDescription:{merged_description}\nGPcreateNetwork:true\nGPcreateMembers:true\nOrigin:merged\n"

    # Create extra lines for NameMethod1 and NameMethod2
    extra_lines = f'NameMethod1: "{name_input1}"\nNameMethod2: "{name_input2}"\n\n'

    # Check if Final_Merged_Model.txt exists
    try:
        with open(output3_path, 'r') as existing_file:
            existing_content = existing_file.read()
    except FileNotFoundError:
        existing_content = ""

    # Open Final_Merged_Model.txt for writing
    with open(output3_path, 'w') as output3_file:
        # Write the merged introduction section to output3.txt
        output3_file.write(merged_introduction)
        output3_file.write(extra_lines)
        output3_file.write(existing_content)

    print("Merged introduction and extra lines added to Final_Merged_Model.txt")

# Specify the paths for input1.txt, input2.txt, and output3.txt
input1_path = "/Users/Documents/Main/Preprocessing/DSL_Model_A.txt"
input2_path = "/Users/Documents/Main/Preprocessing/DSL_Model_B.txt"
output3_path = "/Users/Documents/Main/Postprocessing/Final_Merged_Model.txt"

# Call the function
add_merged_introduction(input1_path, input2_path, output3_path)

Merged introduction and extra lines added to Final_Merged_Model.txt
